# Decision Tree Regression
<hr>
<br>

**`Standart Devation`**

<Br>
$Std = \sqrt{\dfrac{\sum{ (x-myu)^2}} {n}}$
<br>
    <br>
$myu = \dfrac{\sum{x}}{n}$
<br>
<hr>
<br>
    
**`Standart Devation Reduction`**
<br>
<br>
$SDTR = STD(Label) - STD(Feature, Label)$
    

In [4]:
def std(arr):
    myu = arr.mean()
    return np.sqrt(np.sum(np.square(arr - myu))/ len(arr))

In [9]:
def stdr(arr1,arr2):
    std_arr = std(arr2)
    uniq = np.unique(arr1)
    list_stdr = list()
    for i in uniq:
        label = arr2[arr1==i]
        print(i,":", std(label), "\t",len(label))
        list_stdr.append([len(label), std(label)])
    list_stdr = np.array(list_stdr).T
    
    return std_arr - np.dot(list_stdr[0]/len(arr1), list_stdr[1])
    


# Decision Tree Classification
<hr>
<br>

**`Entropy`**

$E(S) = \sum_{i=1}^n (- p_i log_2 p_i)$

<hr>
<br>

**`Information Gain`**
<br>
$Gain(S,A) = E(S) - \sum_{(v Values(A))}  \dfrac{S}{S_v} E(S_v)  $

<hr>
<br>

**`Gini Index`**

$Gini = 1 - \sum_{i=1}^j p_i^2$


In [20]:
def compute_impurity(feature, criteria="entropy"):
    """
    Supported impurity criteria: "entropy", "gini"
    """
    probs = np.unique(feature, return_counts=True)[1] 
    probs = probs / len(feature)
    
    if criteria=="entropy":
        return -1 * np.sum(np.log2(probs) * probs )
    
    elif criteria=="gini":
        
        return 1 - np.sum(np.square(probs))
    
    else:
        raise ValueError("Unkown criteria")

In [150]:
def information_gain(data, target,  split_criterion="entropy"):
    entropy_list = list()
    weight_list = list()
    target_entropy = compute_impurity(target, split_criterion)   
    for i in np.unique(data):
        
        label = target[data == i]
        
        
        entropy = compute_impurity(feature=label,  criteria=split_criterion)
        
        
        entropy_list.append( entropy )
        weight_list.append(len(label)/len(data))
   
    information_gain = target_entropy - np.sum(np.array(entropy_list) * weight_list)
    return information_gain
        

In [151]:
class Node():
    def __init__(self, feature_index=None, threshold=None, left=None, right=None, info_gain=None, value=None):
        ''' constructor ''' 
        
        # for decision node
        self.feature_index = feature_index
        self.threshold = threshold
        self.left = left
        self.right = right
        self.info_gain = info_gain
        
        # Oxirgi chiqish qiymati uchun
        self.value = value

In [154]:
class CustomDecisionTreeClassifier():
    def __init__(self, min_samples_split=2, max_depth=5):
        ''' Konstruktor '''
        
        # Bosh tugunni istalizatsiya qilish
        self.root = None
        
        # Bo'linish chegarasi bilan maksimal chuqurlik
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        
    def build_tree(self, dataset, curr_depth=0):
        ''' Daraxtni qurish uchun rekursiv funksiya''' 
        
        X, Y = dataset[:,:-1], dataset[:,-1]
        num_samples, num_features = np.shape(X)
        
        # To'tatish shartlari bajarilmaguncha bo'linadi
        if num_samples>=self.min_samples_split and curr_depth<=self.max_depth:
            
            # Eng yaxshi bo'linishni topish funksiyasi
            best_split = self.get_best_split(dataset, num_samples, num_features)
            
            # Info gain ni musbatlikka tekshiryapmiz
            if best_split["info_gain"]>0:
                
                # chapga bo'linish
                left_subtree = self.build_tree(best_split["dataset_left"], curr_depth+1)
                
                # onga bo'linish
                right_subtree = self.build_tree(best_split["dataset_right"], curr_depth+1)
                
                # Daraxt tugunlarini qaytaradi
                return Node(best_split["feature_index"], best_split["threshold"], left_subtree, right_subtree, best_split["info_gain"])
        
        # Chiquvchi qiymatnin hisoblash funksiyasi
        leaf_value = self.calculate_leaf_value(Y)
        
        # Chiquvchi qiymatni qaytaradi
        return Node(value=leaf_value)
    
    def get_best_split(self, dataset, num_samples, num_features):
        ''' Eng yaxshi bo'linishni topish funksiyasi '''
        
        # Bo'linishni saqlash uchun fuksiya
        best_split = {}
        max_info_gain = -float("inf")
        
        # Barcha feature ni tekshirish
        for feature_index in range(num_features):
            
            feature_values = dataset[:, feature_index]
            
            thresh_min, thresh_max = np.min(np.unique(feature_values)), np.max(np.unique(feature_values))
            possible_thresholds = np.arange(thresh_min, thresh_max+0.1, 0.1)
            
            
            # Eng yaxshi bo'linishni topish            
            for threshold in possible_thresholds:
                
                # Chegara bo'yicha bo'linishlarni olish                
                dataset_left, dataset_right = self.split(dataset, feature_index, threshold)
                
                # Bo'lingan datalarni tekshirish
                if len(dataset_left)>0 and len(dataset_right)>0:
                    y, left_y, right_y = dataset[:, -1], dataset_left[:, -1], dataset_right[:, -1]
                    
                    # Information Gain ni hisoblash
                    curr_info_gain = self.information_gain(y, left_y, right_y, "gini")
                    
                    # Eng yaxshi bo'linishni saqlash
                    if curr_info_gain>max_info_gain:
                        best_split["feature_index"] = feature_index
                        best_split["threshold"] = threshold
                        best_split["dataset_left"] = dataset_left
                        best_split["dataset_right"] = dataset_right
                        best_split["info_gain"] = curr_info_gain
                        max_info_gain = curr_info_gain
                        
        # Eng yaxshi bo'linishni saqlash
        return best_split
    
    def split(self, dataset, feature_index, threshold):
        ''' Ma'lumotlarni bo'lish funksiyasi '''
        
        dataset_left = np.array([row for row in dataset if row[feature_index]<=threshold])
        dataset_right = np.array([row for row in dataset if row[feature_index]>threshold])
        return dataset_left, dataset_right
    
    def information_gain(self, parent, l_child, r_child, mode="entropy"):
        ''' Information Gain ni hisoblash funksiyasi '''
        
        weight_l = len(l_child) / len(parent)
        weight_r = len(r_child) / len(parent)
        if mode=="gini":
            gain = self.gini_index(parent) - (weight_l*self.gini_index(l_child) + weight_r*self.gini_index(r_child))
        else:
            gain = self.entropy(parent) - (weight_l*self.entropy(l_child) + weight_r*self.entropy(r_child))
        return gain
    
    def entropy(self, y):
        ''' Entropy ni hisoblash funksiyasi '''
        
        class_labels = np.unique(y)
        entropy = 0
        for cls in class_labels:
            p_cls = len(y[y == cls]) / len(y)
            entropy += -p_cls * np.log2(p_cls)
        return entropy
    
    def gini_index(self, y):
        ''' Gini indexni hisoblash fuksiyasi '''
        
        class_labels = np.unique(y)
        gini = 0
        for cls in class_labels:
            p_cls = len(y[y == cls]) / len(y)
            gini += p_cls**2
        return 1 - gini
        
    def calculate_leaf_value(self, Y):
        ''' function to compute leaf node '''
        
        Y = list(Y)
        return max(Y, key=Y.count)
    
    def print_tree(self, tree=None, indent=" "):
        ''' function to print the tree '''
        
        if not tree:
            tree = self.root

        if tree.value is not None:
            print(tree.value)

        else:
            print("X_"+str(tree.feature_index), "<=", tree.threshold, "?", tree.info_gain)
            print("%sleft:" % (indent), end="")
            self.print_tree(tree.left, indent + indent)
            print("%sright:" % (indent), end="")
            self.print_tree(tree.right, indent + indent)
    
    def fit(self, X, Y):
        ''' function to train the tree '''
        
        dataset = np.concatenate((X, Y), axis=1)
        self.root = self.build_tree(dataset)
    
    def predict(self, X):
        ''' function to predict new dataset '''
        
        preditions = [self.make_prediction(x, self.root) for x in X]
        return preditions
    
    def make_prediction(self, x, tree):
        ''' function to predict a single data point '''
        
        if tree.value!=None: return tree.value
        feature_val = x[tree.feature_index]
        if feature_val<=tree.threshold:
            return self.make_prediction(x, tree.left)
        else:
            return self.make_prediction(x, tree.right)